__Note: Run cells in top-down order__

Authors: Kevin Chang, Bharati Malik, Hariz Mohd Rani Hisham, Ritumbhra Sagar

In [1]:
import os
import sqlite3
from sqlite3 import Error
import pandas as pd  # Sorry

### Setting up directories

Below, enter the directory / file path pointing to your climate change folders:

In [ ]:
directory = 'C:/<YOUR FILEPATH>/'  # Dir where all climate change folders live

Then, enter the directory where you want your database to be created:

In [ ]:
db = 'C:/<YOUR FILEPATH>/<YOUR DATABSE NAME>.db'  # Target location for created database

## Part 1: Creating the database and single table

### Define Create Table function

In [3]:
def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        return conn
    except Error as e:
        print(e)
 
    return conn


In [4]:
def create_table(conn, create_table_sql):
    """ create a table from the create_table_sql statement
    :param conn: Connection object
    :param create_table_sql: a CREATE TABLE statement
    :return:
    """
    try:
        c = conn.cursor()
        c.execute(create_table_sql)
    except Error as e:
        print(e)


In [6]:
def main():
    
    sql_create_weather_table = """ CREATE TABLE IF NOT EXISTS weather (
                                        region text NOT NULL,
                                        latitude float NOT NULL,
                                        longitude float NOT NULL,
                                        date text,
                                        precipitation float,
                                        max_temp float,
                                        min_temp float,
                                        wind float
                                    ); """
 
    # create a database connection
    conn = create_connection(db)
 
    # create tables
    if conn is not None:
        # create weather table
        create_table(conn, sql_create_weather_table)
        conn.close()
    else:
        print("Error! cannot create the database connection.")
 
 
if __name__ == '__main__':
    main()

## Part 2: Extracting data from text files and inserting data into database table

### Define date extraction functions

In [7]:
def extract_date(year, month):
    date = None
    try:
        date = (year + '-' + month,)
        return date
    except:
        print("Enter a valid year / month format!")
    return date

def replace_date(line, new_date):
    new_line = None
    try:
        new_line = new_date + line[2:]
        return new_line
    except:
        print("Check that a valid record and date format is passed in.")
    return new_line

### Define INSERT statements

In [8]:
def insert_data(chunk,database):
    try:
        # create a database connection
        conn = create_connection(database)
        cur = conn.cursor()
        with conn:
            cur.executemany("INSERT INTO weather VALUES (?,?,?,?,?,?,?,?)", chunk)
        conn.commit()
        conn.close()
    except:
        print("Check that record passed in is formatted correctly!")

### Define Data Processing functions

In [9]:
def name_split(filename):
    try:
        if len(filename) > 1:
            lat_long = filename.split('_')
            return lat_long[1], lat_long[2]
    except:
        print("Check filename is correct!")
        
def combine_tuple(tup1, tup2):
    comb = None
    try:
        comb = tup1 + tup2
        return comb
    except:
        print("Check tuples passed in are correct!")
        
def row_split(cont):
    measurements = None
    try:
        rows = cont.split('\n')
        rows = rows[:-1]
        measurements = [tuple(x.split()) for x in rows]
        return measurements
    except:
        print("Check that file contents are correct!")


In [10]:
def process_file(filename, region, folder_dir):
    data_chunk = None
    try:
        if filename.startswith('data'):

            lat_long = name_split(filename)
            region_co = combine_tuple(region, lat_long)

            with open(folder_dir + '//' + filename, 'r') as f:
                cont = f.read()

            measurements = row_split(cont)

            data_chunk = [combine_tuple(region_co, replace_date(x, extract_date(x[0], x[1]))) for x in measurements]
            return data_chunk
        else:
            return data_chunk
    except:
        print("Data insertion failed.")

In [11]:
def main(directory, database):
    counter = 0
    for folder in os.listdir(directory):
        print(str(folder) + " folder in progress.")
        try:
            
            # Assumes that regional folders are formatted in short-form code
            
            if len(folder) < 7:
                folder_dir = str(directory) + str(folder)
                region = (str(folder),)
                
                for filename in os.listdir(folder_dir):
                    insert_data(process_file(filename, region, folder_dir), database)
                    counter += 1

                    if (counter % 1000) == 0:
                        print("Still working...")           
                        continue
                    else:
                        continue     
            else:
                continue
        except:
            print("You broke the Internet.")
            
        print(str(folder) + " folder done.")
        
    print("Congratulations, Mr. Stark. All data successfully extracted from all folders.")


### Run Main function

In [ ]:
if __name__ == '__main__':
    main(directory, db)